# Settings

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from bisect import bisect
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
##################
# FULL OPA
##################

downpayment = 0.20     # percent of price
interest = 0.055       # percent of price
mortgage_duration = 30 # years
rofr = 0.1             # percent reduction in sales price
spillover = 0.1        # percent of sales that don't go through
off_market = 0.1       # percent of sales off market
dti = 0.36             # debt-to-income ratio that's required to get the mortgage approved
ami = 149_600          # average median income
sfh = 1                # include single family homes in OPA? 1 = yes, 0 = no

# Read data

In [3]:
df = pd.read_csv('final_results_1000.csv')

In [4]:
df.columns = [c.lower().replace(' ', '_') for c in df.columns]

n = len(df)
sim_ids = set(df.simulation)
num_sims = len(sim_ids)
num_per_sim = n / num_sims

# Computation

In [5]:
def get_category_coarse(x):
    idx = bisect([80, 160], x)
    cats = ['Low Income', 'Middle Income', 'Upper Income']
    return cats[idx]

df['p_ami'] = df.income / ami * 100
df['mortgage'] = (1 - downpayment) * ((1 + interest) ** mortgage_duration) * (1 - rofr) * df.price / mortgage_duration
df['viable'] = (df.mortgage / df.income <= dti) 

# 0 = not eligible, 1 = still eligible
df['eligible_spillover'] = 1 - (np.random.random(n) < spillover)
df['eligible_market'] = 1 - (np.random.random(n) < off_market)
df['eligible_sfh'] = np.ones(n) if sfh == 1 else (df.property_indicator != 10) * 1

df['viable_eligible'] = df.eligible_spillover * df.eligible_market * df.eligible_sfh * df.viable
df['income_category'] = df.p_ami.apply(get_category_coarse)


# Results

In [6]:
ve = df.groupby('simulation').mean().viable_eligible
m, s = ve.mean(), ve.std()

print(f'viable & eligible: {m:.2%}')
print(f'95% confidence interval (viable & eligible): {m - 2*s:.2%}, {m + 2*s:.2%}')

viable & eligible: 8.56%
95% confidence interval (viable & eligible): 6.46%, 10.65%
